# Inference of mutational signatures using Gibbs sampling
---

# Setup

In [ ]:
import pandas as pd
import arviz as az
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from time import time
from utils import *

In [ ]:
df = pd.read_csv("../../../resources/NikZainal-breast_cancer.csv", sep="\t", 
        usecols=["sample_id","5_prime_flanking_sequence_in_pyrimidine_context","ref_base_pyrimidine_context",
                    "mutant_base_pyrimidine_context","3_prime_flanking_sequence_in_pyrimidine_context"])
df["substitution_type"] = (df["5_prime_flanking_sequence_in_pyrimidine_context"].str[-1] + 
                            "[" + df["ref_base_pyrimidine_context"] + 
                            ">" + df["mutant_base_pyrimidine_context"] +
                            "]" + df["3_prime_flanking_sequence_in_pyrimidine_context"].str[0])
df

In [ ]:
counts = pd.crosstab(df["sample_id"], df["substitution_type"])
counts

In [ ]:
sns.clustermap(counts.div(counts.sum(axis=1), axis=0), robust=True, cmap="Blues")

# Sampling

In [ ]:
start = time()
set_srng(seed=1234)
mcmc_results = sample_fixed_hyperparams(counts, mcmc_steps=100000)
mcmc_time = time() - start

In [ ]:
print(f"MCMC time: {mcmc_time:.1f}s")

In [ ]:
az.to_netcdf(mcmc_results, "../../../results/signeR-replication/fixed_hyperparams.netcdf")

In [ ]:
mcmc_results = az.from_netcdf("../../../results/signeR-replication/fixed_hyperparams.netcdf")

# Effective sample size

In [ ]:
az.ess(mcmc_results).median()

In [ ]:
az.ess(mcmc_results).min()

In [ ]:
az.ess(mcmc_results).max()

In [ ]:
az.ess(mcmc_results).median()/mcmc_time

In [ ]:
az.ess(mcmc_results).min()/mcmc_time

In [ ]:
az.ess(mcmc_results).max()/mcmc_time

# Point estimates: Signature spectra

In [ ]:
breast_sigs = mcmc_results.posterior.mean(["chain","draw"]).signature.to_pandas()
breast_sigs

In [ ]:
sns.clustermap(breast_sigs, cmap="Blues")

In [ ]:
def rank(series):
    sorted_series = series.sort_values(ascending=False)
    return sorted_series.index + ": " + sorted_series.map("{:.1%}".format).values

breast_sigs.T.apply(rank).head(5).reset_index().drop(columns="substitution")

# Point estimates: Signature exposures

In [ ]:
expected_exposures = mcmc_results.posterior.mean(["chain","draw"]).exposure.to_pandas()
sns.clustermap(expected_exposures, cmap="Greens")

In [ ]:
az.plot_forest(
    mcmc_results.posterior.exposure.mean("specimen"),
    var_names=["exposure"], 
    kind='ridgeplot', 
    ridgeplot_quantiles=[0.5],
    hdi_prob=0.99,
    ridgeplot_overlap=3)

# Comparison to COSMIC signatures

In [ ]:
cosmic_sigs = pd.read_csv("../../../resources/COSMIC_v3.2_SBS_GRCh37.txt", sep="\t", index_col="Type")
cosmic_sigs = cosmic_sigs.loc[breast_sigs.columns]

In [ ]:
def l2_norms(df):
    return df.apply(np.linalg.norm).values

cosine_sim = breast_sigs @ cosmic_sigs / (l2_norms(breast_sigs.T)[:, None] * l2_norms(cosmic_sigs)[None, :])
cosine_sim = cosine_sim.applymap("{:.3f}".format)
cosine_sim_long = cosine_sim.reset_index().melt("sig").sort_values(by="value", ascending=False)
cosine_sim_long.columns = ["Signature (repl.)", "Signature (COSMIC)", "Similarity"]
cosine_sim_long.groupby("Signature (repl.)").head(1).reset_index().drop(columns="index")

## Is S4 driven by spontaneous deamination?

In [ ]:
CpG = mcmc_results.posterior.substitution.str.contains("\[C>T\]G")
S4_CpG_proportion = (
    mcmc_results.posterior
    .signature
    .sel(sig="S4", substitution=CpG)
    .sum("substitution")
    .values
    .flatten()
)

In [ ]:
sns.displot(S4_CpG_proportion, aspect=2)
plt.axvline(S4_CpG_proportion.mean(), ls="--", c="k", label="Posterior mean")
plt.axvline(4/96, ls="--", c="gray", label="Prior mean")
plt.axvline(cosmic_sigs.loc[cosmic_sigs.index.str.contains("\[C>T\]G"), "SBS1"].sum(), ls="--", c="r", label="COSMIC SBS1")
plt.xlim(0,1)
plt.yticks([])
plt.ylabel("")
plt.xlabel("Proporção de desaminação CpG na assinatura S4")
plt.legend();